In [0]:
import numpy as np
import tensorflow as tf

In [1]:
from keras.datasets import cifar10

(train_x, train_y), (test_x, test_y) = cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 4s 0us/step


In [0]:
num_classes = 10
batch_size = 32
epochs = 60

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 0.2,
    width_shift_range = 0.3,
  horizontal_flip = True,
  rescale = 1./255.
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255.
)

train_y_one = tf.keras.utils.to_categorical(train_y)
test_y_one = tf.keras.utils.to_categorical(test_y)

train_generator = train_datagen.flow(train_x, train_y_one, shuffle = True)
test_generator = test_datagen.flow(test_x, test_y_one)

In [0]:
class CNN(tf.keras.Model):
  def __init__(self, num_classes):
    super(CNN, self).__init__()
    layers = tf.keras.layers
    self.layer_list = []
    self.layer_list.append(layers.Conv2D(32, (3,3), padding = 'SAME', activation = 'relu', input_shape = train_x.shape[1:]))
    self.layer_list.append(layers.Conv2D(32, (3,3), padding = 'SAME', activation = 'relu'))
    self.layer_list.append(layers.MaxPool2D())
    self.layer_list.append(layers.Dropout(0.25))
    self.layer_list.append(layers.Conv2D(64, (3,3), padding = 'SAME', activation = 'relu'))
    self.layer_list.append(layers.Conv2D(32, (3,3), padding = 'SAME', activation = 'relu'))
    self.layer_list.append(layers.MaxPool2D())
    self.layer_list.append(layers.Dropout(0.25))
    self.layer_list.append(layers.Flatten())
    self.layer_list.append(layers.Dense(512, activation = 'relu'))
    self.layer_list.append(layers.Dropout(0.5))
    self.layer_list.append(layers.Dense(num_classes, activation = 'softmax'))
  
  def __call__(self, x, training = False, mask = None):
    for layer in self.layer_list:
      x = layer(x)
    return x

In [0]:
model = CNN(num_classes)

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(0.001),
    metrics = ['accuracy']
)

In [0]:
model.fit_generator( 
    train_generator, len(train_x)//batch_size, epochs = epochs,
    validation_data = test_generator, validation_steps = len(test_x)//batch_size
)